In [144]:
# imports
import requests
import os
import pandas as pd
import numpy as np

#read in the data from part 1 and store it as a Data Frame
bike_stations_df = pd.read_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/statistical_modeling/data/stations_data.csv')
bike_stations_df.head()

,name,latitude,longitude,free bikes
0,Lille Schous plass,59.920259,10.760629,6
1,Sjølyst,59.921673,10.676660,15
2,Tøyenparken,59.915667,10.777567,4
3,Lindern,59.935888,10.735006,0
4,Uelands gate,59.929545,10.748986,10


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [180]:
def get_poi(latitude, longitude):
    api_key = os.environ["FOURSQUARE_API_KEY"]
    base_url = "https://api.foursquare.com/v3/places/search?"
    radius = 1000
    category = 13065  # Restaurant category
    #set the fields to be returned by the request
    fieldnames = [
        'categories',
        'name',
        'location',
        'geocodes',
        'tel',
        'website',
        'hours',
        'rating',
        'popularity',
        'price',
        'store_id'
    ]
    headers = {
        "accept": "application/json",
        "Authorization": api_key
    }
    
    # Ensure fields are joined correctly
    fields = '%2C'.join(fieldnames)
    
    # Construct the API request URL
    url = f"{base_url}ll={latitude}%2C{longitude}&radius={radius}&categories={category}&fields={fields}"

    # a session object allows certain parameters and settings to persist
    # across multiple requests made with that session
    with requests.Session() as session:
        try:
            # Send the request and handle errors
            response = session.get(url, headers=headers)
            response.raise_for_status()
            
            # Parse the JSON response
            data = response.json()
            return data
        except requests.exceptions.RequestException as e:
            print(f"Request Error: {e}")
        except ValueError as e:
            print(f"Error parsing response JSON: {e}")
    return None


def get_poi_data(data_frame):
    locations = data_frame[['latitude', 'longitude']]
    
    # Apply get_poi function to each row in the DataFrame
    pois_list = locations.apply(lambda row: get_poi(row['latitude'], row['longitude']), axis=1)
    
    # Convert the resulting Series to a list
    return pois_list.to_list()

In [181]:
#return the json data in list form
poi_data = get_poi_data(bike_stations_df)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [196]:
#generate the keys from the json data
poi_data[0]['results'][0].keys()


dict_keys(['categories', 'geocodes', 'hours', 'location', 'name', 'popularity', 'price', 'rating', 'tel', 'website'])

Put your parsed results into a DataFrame

In [197]:
#create the database
data_list = {
    'id': [],
    'name': [],
    'description': [],
    'location': [],
    'latitude': [],
    'longitude': [],
    'tel': [],
    'website': [],
    'hours': [],
    'rating': [],
    'store_id': []
}

for result in poi_data:
    data_list['id'].append(result.get('id', None))
    data_list['name'].append(result.get('name', None))
    data_list['location'].append(result.get('location', None))
    data_list['latitude'].append(result['geocodes']['main'].get('latitude', None))
    data_list['longitude'].append(result['geocodes']['main'].get('longitude', None))
    data_list['tel'].append(result.get('tel', None))
    data_list['website'].append(result.get('website', None))
    data_list['rating'].append(result.get('rating', None))
    data_list['store_id'].append(result.get('store_id', None))

data_list

KeyError: 'geocodes'

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [115]:
def yelp_poi(longitude, latitude):
    base_url = "https://api.yelp.com/v3/businesses/search?"
    radius = 1000
    category = 'restaurant'
    url = f"{base_url}latitude={latitude}&longitude={longitude}&radius={radius}&categories={category}"

    headers = {"accept": "application/json"}

    # a session object allows certain parameters and settings to persist
    # across multiple requests made with that session
    with requests.Session() as session:
        #ensure the request is valid
        try:
            response = session.get(url, headers=headers)
            response.raise_for_status()
            data = response.json()
            return data
        except requests.exceptions.RequestException as e:
            print(f"Request Error: {e}")
            return None
        except ValueError as e:
            print(f"Error parsing response JSON: {e}")
            return None

def yelp_poi_data(data_frame):
    locations = data_frame[['latitude', 'longitude']]
    num_poi = locations.apply(lambda row: get_poi(row['latitude'], row['longitude']), axis=1)
    return num_poi.to_list()



Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [116]:
poi_data_yelp = yelp_poi_data(bike_stations_df)

In [127]:
business_id = poi_data_yelp[0]['results'][0].get('fsq_id')
business_id

'57f53ed7498e23279caaed8e'

Put your parsed results into a DataFrame

In [ ]:
#insert data frame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating